# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Borja Allo Casal   <br>
Url: https://github.com/zetaholic/SEMINARIO <br>
(el enlace lleva al repositorio, donde estará la entrega de la primera convocatoria y esta: "Poyecto_Algoritmos_C2_Borja_Allo")<br>

Problema:

>3. Combinar cifras y operaciones

Descripción del problema:<br>
El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva:<br>
• Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/).<br>
• Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una
cantidad dada. Un ejemplo sería para obtener el 4:<br>

"4 + 2 - 6 / 3 * 1 = 4"

....

(*) La respuesta es obligatoria



---



                                        

In [ ]:
# Importaciones y librerías a emplear

import math
import itertools
import random
import time
from fractions import Fraction



---


###(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>


###¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?




**Pregunta obligatoria:**<br>
Sin restricciones, se permiten repetir dígitos y operadores libremente. <br>
Pero vamos a considerar la estructura del ejemplo tipo:
- $ d | o | d | o | d | o | d | o | d$

Siendo:
- d = dígito
- o = operador

5 posiciones para cada dígito (siendo estos del 1 al 9): $9^5$ <br>
4 posiciones para los operadores (siendo 4): $4^4$ <br>

- Posibilidades totales: $9^5 . 4^4 = 15116544$

<br>

**Pregunta opcional:**<br>

Al tener en cuenta las restricciones:<br>

* Dígitos: 5 distintos importando el orden.<br>
   * Permutaciones $P(9,5)= 9.8.7.6.5$

* Operadores: usar cada uno una sola vez.
   * $4! = 24$
<br>

Total de posibilidades: $ P(9,5) . 4! = 362880$

In [ ]:
# Respuesta a la pregunta obligatoria

cifras_totales = 9 ** 5 # 5 posiciones por dígito entre 1 y 9
operadores_totales = 4 ** 4 # 4 posiciones para 4 operadores (no importa el orden)
posibilidades = cifras_totales * operadores_totales

print(f"Total de expresiones sin restricciones: {cifras_totales} x {operadores_totales} = {posibilidades}")


Total de expresiones sin restricciones: 59049 x 256 = 15116544


In [ ]:
# Respuesta a la pregunta opcional

combinatoria_cifras = math.perm(9, 5) # Permutaciones P(9,5)
permutaciones_operadores = math.factorial(4)

# Total de posibilidades
total_posibilidades = combinatoria_cifras * permutaciones_operadores

print(f"Número total de expresiones posibles cumpliendo restricciones: {total_posibilidades}")


Número total de expresiones posibles cumpliendo restricciones: 362880




---


###Modelo para el espacio de soluciones<br>
###(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo)


**Pregunta obligatoria:**<br>

La estructura de datos que mejor se adapta al problema es el árbol binario de expresiones, ya que permite representar de forma natural la alternancia entre operadores y dígitos, respetar la jerarquía de operaciones y aplicar técnicas de backtracking y poda sobre un espacio de búsqueda. Aunque inicialmente se podría pensar en una lista por su simplicidad, al avanzar en el diseño se ve que el árbol ofrece una mayor flexibilidad y eficiencia para generar y evaluar expresiones en este problema.
<br>

Una expresión aritmética se representa muy bien como un árbol binario:

* los dígitos en las hojas

* los operadores en los nodos internos

Ventajas:

* Respeta de forma natural la jerarquía de operaciones

* Permite aplicar backtracking o recorrido recursivo para evaluar

* Facilita la poda de ramas no prometedoras

In [ ]:
# PARA EL EJEMPLO DEL ENUNCIADO

# Definimos una clase para los nodos del árbol de expresión
class Nodo:
    def __init__(self, valor, izq=None, der=None):
        self.valor = valor   # operador o dígito
        self.izq = izq       # subárbol izquierdo
        self.der = der       # subárbol derecho

# Construimos el árbol para la expresión: 4+2-6/3*1
# Nota: Python sigue precedencia: * y / antes que + y -
# Así que realmente es: ( (4+2) - ( (6/3) * 1 ) )

arbol = Nodo('-',
             Nodo('+',
                  Nodo('4'),
                  Nodo('2')),
             Nodo('*',
                  Nodo('/',
                       Nodo('6'),
                       Nodo('3')),
                  Nodo('1'))
            )

# Función para imprimir el árbol en notación prefija (operador primero)
def imprimir_prefijo(nodo):
    if nodo is None:
        return ""
    return f"{nodo.valor} {imprimir_prefijo(nodo.izq)} {imprimir_prefijo(nodo.der)}"

# Función para imprimir en notación infija (forma "humana", la nuestra)
def imprimir_infijo(nodo):
    if nodo.izq is None and nodo.der is None:
        return nodo.valor
    return f"({imprimir_infijo(nodo.izq)} {nodo.valor} {imprimir_infijo(nodo.der)})"

print("Forma prefija:", imprimir_prefijo(arbol))
print("Forma infija:", imprimir_infijo(arbol))



Forma prefija: - + 4   2   * / 6   3   1  
Forma infija: ((4 + 2) - ((6 / 3) * 1))




---


###Según el modelo para el espacio de soluciones<br>
###(*)¿Cual es la función objetivo?

###(*)¿Es un problema de maximización o minimización?

**Pregunta obligatoria:**<br>

####La función objetivo:

En el problema, cada expresión generada (con dígitos y operadores) se puede evaluar y produce un resultado numérico.

La *función objetivo* consiste en medir cómo de cerca está ese resultado de una cantidad objetivo dada (un “número objetivo”, como el ejemplo del enunciado).

Si:

- $E$ es una expresión,  
- $f(E)$ es el valor numérico de esa expresión,  
- $K$ es el valor objetivo,

entonces la función objetivo puede definirse como:

$$
F(E) = |f(E) - K|
$$

Es decir, la **distancia absoluta** entre el valor de la expresión y el valor buscado.

<br>


**Pregunta obligatoria:**<br>

####¿Maximización o minimización?

Es un **problema de minimización**, ya que lo que buscamos es reducir al mínimo la diferencia entre el resultado de la expresión y el valor objetivo $K$.

Si encontramos una expresión exacta, es decir:

$$
f(E) = K
$$

entonces la función objetivo se convierte en:

$$
F(E) = |f(E) - K| = 0
$$

lo cual representa el **valor mínimo posible**.

<br>

En caso de que no exista una solución exacta, lo que buscaríamos sería la *mejor aproximación*, es decir, aquella expresión cuya evaluación produzca el **menor error** posible.

Pero, una vez resuelto el problema completo, se observa que entre los valores mínimo (-69) y máximo (77), se consiguen **todos** los valores enteros disponibles en ese rango.


In [ ]:
# Voy a generar un ejemplo con una muestra aleatoria
digitos = [str(i) for i in range(1, 10)] # Todas las crifras del 1 al 9
operadores = ['+', '-', '*', '/']

resultados_enteros = set()
muestras = 10000  # Para el ejemplo vamos a utilizar 10000 muestras. Pero lo dejo con opción de subir o bajar el número de muestras para pruebas

def evaluar(expr):
    try:
        resultado = eval(expr) # Uso eval(), pero aplicando el filtro siguiente
        if resultado == int(resultado):  # Solo enteros
            return int(resultado)
    except ZeroDivisionError: # Divisiones entre 0. A priori no debería hacer falta. No hay posibilidades de divisiones entre 0; pero por si acaso hay algún error
        return None
    return None

for _ in range(muestras): # Genero las entradas aleatorias
    cifras = random.sample(digitos, 5)
    ops = random.sample(operadores, 4)
    expr = ""
    for i in range(4): # Inicio bucle para ir construyendo la expresión generada en un string
        expr += cifras[i] + ops[i]
    expr += cifras[4]
    resultado = evaluar(expr) # Uso la función creada (que es como una modificación del "eval()", pero incluye la "criba" de las divisiones entre 0)
    if resultado is not None:
        resultados_enteros.add(resultado) # Voy añadiendo los resultados enteros

# Análisis de resultados
minimo = min(resultados_enteros)
maximo = max(resultados_enteros)
faltantes = [i for i in range(minimo, maximo + 1) if i not in resultados_enteros] # Compruebo los faltantes entre el mín. y el máx.

print(f"Valores enteros encontrados: {len(resultados_enteros)}")
print(f"Valor mínimo: {minimo}")
print(f"Valor máximo: {maximo}")
print(f"¿Todos los enteros entre mínimo y máximo están presentes?: {len(faltantes) == 0}")
if faltantes:
    print(f"Valores faltantes: {faltantes[:10]}{'...' if len(faltantes) > 10 else ''}")


Valores enteros encontrados: 139
Valor mínimo: -66
Valor máximo: 76
¿Todos los enteros entre mínimo y máximo están presentes?: False
Valores faltantes: [-65, -60, -54, -53]


En este ejemplo con una muestra aleatoria no se consiguen todos los valores enteros en el rango del mínimo al máximo hallado. Pero, evaluando todas las expresiones posibles, sí se consiguen.



---


###Diseña un algoritmo para resolver el problema por fuerza bruta

**Pregunta opcional:**<br>

Diseñar un algoritmo por fuerza bruta supone explorar y evaluar **todas** las expresiones posibles (dentro de las restricciones) y almacenar aquellas que sean válidas (que cumplan el requisito de ser un número entero).<br>
Sería como llevar el ejemplo con muestras limitadas que hice antes, pero evaluando todas las opciones existentes.<br>
Pasos que he intentado seguir:

1.   Generar todas las combinaciones posibles de 5 cifras (escogiendo números entre el 1 y el 9).
2.   Para cada combinación de las cifras: generar todas las permutaciones u ordenaciones posibles de ellos (orden de aparición en la expresión).
3.   Para cada ordenación de cifras, generar todas las permutaciones u ordenaciones posibles de los 4 operadores disponibles.
4.   Construir la expresión final: con la estructura "cifra 1, operador 1, cifra 2, operador 2,..., operador 4, cifra 5".
5.   Evaluar la expresión: sin divisiones entre 0 y que sean enteros.
6.   Guardar los resultados que cumplan.

Por fuerza bruta se considerarán todas las expresiones, pero la ejecución del código es muy exigente.

In [ ]:
# Diseño por FUERZA BRUTA

digitos = [str(i) for i in range(1, 10)]
operadores = ['+', '-', '*', '/']

expresiones_validas = [] # Voy a guardar todas las expresiones generadas que den un número entero
r_enteros = [] # Voy a guardar todos los enteros generados

def evaluar(expr): # Igual que hice en el ejemplo con muestra aleatoria
    try:
        resultado = eval(expr)
        if resultado == int(resultado):
            return int(resultado)
    except ZeroDivisionError:
        return None
    return None

# Todas las combinaciones posibles
for cifras in itertools.combinations(digitos, 5): # Combinatoria "9 sobre 5"
    for perm_cifras in itertools.permutations(cifras): # Permutaciones de las cifras anteriores
        for perm_ops in itertools.permutations(operadores): # Permutaciones de los operadores
            expr = "" # Donde almacenaré la expresión dentro del bucle
            for i in range(4):
                expr += perm_cifras[i] + perm_ops[i]
            expr += perm_cifras[4]
            resultado = evaluar(expr)
            if resultado is not None:
                expresiones_validas.append((expr, resultado)) # Mantengo expresión y resultado asociados
                r_enteros.append(resultado) # Me vale para calcular después el máximo y el mínimo


print(f"Total de expresiones válidas: {len(expresiones_validas)}") # Muestra todas las expresiones que den números enteros que se hayan generado
print(f"Valor mínimo: {min(r_enteros)}")
print(f"Valor máximo: {max(r_enteros)}")
print(f"Número de enteros distintos: {len(list(set(r_enteros)))}")
print("Ejemplos:")
for expr, res in expresiones_validas[9:20]: # Para enseñar algunos de los resultados válidos obtenidos, escojo una muestra entre la pos. 9 y la 19
    print(f"{expr} = {res}")

Total de expresiones válidas: 90000
Valor mínimo: -69
Valor máximo: 77
Número de enteros distintos: 147
Ejemplos:
1-3*4/2+5 = 0
1*3+4/2-5 = 0
1*3-4/2+5 = 6
1+3-4*5/2 = -6
1-3+4*5/2 = 8
1+3-5/2*4 = -6
1-3+5/2*4 = 8
1+3-5*4/2 = -6
1+3*5-4/2 = 14
1-3+5*4/2 = 8
1-3*5+4/2 = -12


Del algoritmo por fuerza bruta podemos considerar como verdaderos los resultados obtenidos:
* Expresiones válidas, es decir, que den lugar a un entero: 90000 (aunque hay expresiones que dan el mismo entero).
* Valor máximo hallado: -69
* Valor mínimo hallado: 77
* Nº de enteros distintos: 147
<br>

Si el menor es $-69$, el mayor es $77$ y tenemos $147$ valores distintos; entonces podemos deducir que se pueden conseguir todos los valores enteros intermedios entre el máximo y el mínimo.



---


###Calcula la complejidad del algoritmo por fuerza bruta

**Pregunta opcional:**

La complejidad del algoritmo varía en función de lo que se entienda por el tamaño de entrada $n$. Aunque, siempre que se utilice el mismo criterio, se puede utilizar de forma generalizada para comparar los algoritmos.

### Opción 1: $n$ es el número de dígitos disponible

Es decir, tenemos los $4$ operadores del enunciado, y hay que escoger $5$ dígitos de los $n$ disponibles.

### Conteo de operaciones elementales:

La complejidad del algoritmo se deriva del número de veces que se ejecutan los bucles anidados.  
Cada uno de estos bucles se basa en funciones de `itertools`.

#### Bucle `itertools.combinations(digitos, 5)`

Este bucle genera todas las combinaciones posibles de 5 dígitos a partir de un conjunto de $n$ dígitos.

El número de combinaciones es:

$$
\binom{n}{5} = \frac{n!}{5!(n-5)!}
$$

Para valores grandes de $n$, el término dominante de esta expresión es $n^5$.  
Por lo tanto, esta operación tiene una complejidad de **$O(n^5)$**.


#### Bucle `itertools.permutations(cifras)`

Este bucle genera las permutaciones de 5 elementos.

El número de permutaciones de 5 elementos es:

$$
5! = 120
$$

Este es un valor constante, no depende de $n$.  
Este bucle se ejecuta una vez por cada combinación del bucle exterior, es decir, $\binom{n}{5}$ veces.


#### Bucle `itertools.permutations(operadores)`

Similar al anterior, este bucle genera las permutaciones de 4 operadores.

El número de permutaciones de 4 operadores es:

$$
4! = 24
$$

También una constante.  
Este bucle se ejecuta $\binom{n}{5} \times 5!$ veces.


#### Bucle interno `for i in range(4)`

Este bucle se ejecuta un número fijo de veces (4).

El número total de iteraciones de este bucle es:

$$
\binom{n}{5} \times 5! \times 4!
$$


#### Función `evaluar(expr)`

Esta función se llama en la iteración más interna.  
El tiempo de ejecución de `eval()` depende de la longitud de la expresión,  
pero dado que la expresión siempre tiene 9 caracteres (5 dígitos y 4 operadores),  
el tiempo de ejecución es constante: **$O(1)$**.


### Simplificación a Notación Big O

Para determinar la complejidad total, nos enfocamos en el término que crece más rápido a medida que $n$ aumenta.  
En este caso, el número total de operaciones está dominado por el primer bucle anidado, cuya complejidad es **$O(n^5)$**.

El número de operaciones es proporcional a:

$$
\binom{n}{5} = \frac{n(n-1)(n-2)(n-3)(n-4)}{120}
$$

Al ignorar las constantes y los términos de menor orden, la complejidad asintótica se simplifica a:

$$
\boxed{O(n^5)}
$$


### Conclusión

La complejidad del algoritmo, asumiendo que el número de dígitos $n$ puede variar, es **$O(n^5)$**.





### Opción 2: la variable de entrada $n$ es el número de operadores

El conjunto de dígitos disponibles es fijo (del 1 al 9). Pero como el número de dígitos a escoger sería $n+1$, si el número $n$ de operadores fuese mayor que 8, no podríamos fijar los valores a escoger del 1 al 9.

El algoritmo explora **todas las combinaciones posibles sin optimizaciones**.  
El cálculo de su complejidad se basa en el **producto del número de opciones en cada etapa**.

#### Paso 1: Permutación de los dígitos

La expresión debe tener $n + 1$ dígitos.  
Estos dígitos, de los cuales se eligen, a priori está limitado a 9.
El número de formas de **elegir y ordenar** $n + 1$ dígitos es el número de permutaciones sin repetición:

$$
 \frac{9!}{(9 - (n + 1))!} = \frac{9!}{(8 - n)!}
$$

Esta función **crece de forma factorial**.

#### Paso 2: Combinación con repetición de los operadores

La expresión debe tener $n$ operadores.  
Como no se pueden repetir, deben elegirse de un conjunto de, al menos, $n$ operadores únicos: para el caso más simple, si el conjunto tiene $n$ elementos, el número de permutaciones es ${n!}$.

**Crece de forma factorial**.

#### Conclusión

Ambos factores tienen un crecimiento factorial. Por lo tanto, podemos decir que su complejidad es:

$$
\boxed{O(n!)}
$$




---



###(*)Diseña un algoritmo que mejore la complejidad del algoritmo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

**Pregunta obligatoria:**<br>

Una mejora al algoritmo de fuerza bruta es aplicar la técnica de vuelta atrás (backtracking) con poda.<br>
El algoritmo construye la expresión paso a paso (dígitos y operadores), evaluando parcialmente en cada etapa. Si detecta que el valor parcial ya no puede conducir a una solución válida (por ejemplo, supera el rango máximo o mínimo posible), descarta esa rama y evita seguir generando expresiones inútiles (expresiones que ya ni siquiera se evaluarán ni tendrán en cuenta, ejecutando menos operaciones).

In [ ]:
# ALGORITMO BACKTRACKING CON PODA

def backtracking_con_poda_conmut():
    digitos = [str(i) for i in range(1, 10)]
    operadores = ['+', '-', '*', '/']

    expresiones_validas = []
    enteros_alcanzados = set()

    comprobaciones = 0   # parciales + finales
    ramas_podadas = 0

    def es_entero(frac):
        return frac.denominator == 1

    def backtrack(expr, usados_digitos, usados_operadores, esperando_operador, ultimo_valor=None, ultimo_dig=None):
        nonlocal comprobaciones, ramas_podadas

        # Caso base: todos los dígitos y operadores usados
        if len(usados_digitos) == 5 and len(usados_operadores) == 4:
            comprobaciones += 1
            try:
                val = eval(expr)
                if val == int(val):
                    entero = int(val)
                    enteros_alcanzados.add(entero)
                    expresiones_validas.append((expr, entero))
            except ZeroDivisionError:
                return
            return

        if esperando_operador:
            for op in operadores:
                if op in usados_operadores:
                    continue
                backtrack(expr + op, usados_digitos, usados_operadores | {op}, False, ultimo_valor, ultimo_dig)

        else:
            for d in digitos:
                if d in usados_digitos:
                    continue

                # Poda 1: evitar simetrías en + y *
                if expr and expr[-1] in ['+', '*'] and ultimo_dig is not None:
                    if int(d) < int(ultimo_dig):
                        ramas_podadas += 1
                        continue

                nueva_expr = expr + d
                try:
                    comprobaciones += 1  # cada eval parcial
                    val = eval(nueva_expr)

                    # Poda 2: valor demasiado grande
                    if abs(val) > 10000:
                        ramas_podadas += 1
                        continue
                    # Poda 3: división no entera ya cerrada
                    frac = Fraction(val).limit_denominator()
                    if '/' in nueva_expr and not es_entero(frac):
                        ramas_podadas += 1
                        continue
                except ZeroDivisionError:
                    ramas_podadas += 1
                    continue

                backtrack(nueva_expr, usados_digitos | {d}, usados_operadores, True, val, d)

    start = time.time()
    backtrack("", set(), set(), False)
    tiempo = time.time() - start

    # Resultados
    if enteros_alcanzados:
        minimo = min(enteros_alcanzados)
        maximo = max(enteros_alcanzados)
    else:
        minimo, maximo = None, None

    print("=== BACKTRACKING CON PODA ===")
    print(f"Tiempo: {tiempo:.2f} s")
    print(f"Valor mínimo: {minimo}")
    print(f"Valor máximo: {maximo}")
    print(f"Enteros distintos: {len(enteros_alcanzados)}")
    print(f"Expresiones válidas: {len(expresiones_validas)}")
    print(f"Comprobaciones totales (parciales+finales): {comprobaciones}")
    print(f"Ramas podadas: {ramas_podadas}")

    return {
        "minimo": minimo,
        "maximo": maximo,
        "enteros": enteros_alcanzados,
        "expresiones_validas": expresiones_validas,
        "comprobaciones": comprobaciones,
        "ramas_podadas": ramas_podadas,
        "tiempo": tiempo
    }


# Ejecutación
if __name__ == "__main__":
    backtracking_con_poda_conmut()


=== BACKTRACKING CON PODA ===
Tiempo: 0.81 s
Valor mínimo: -69
Valor máximo: 77
Enteros distintos: 147
Expresiones válidas: 25570
Comprobaciones totales (parciales+finales): 82228
Ramas podadas: 35753


El diseño final del algoritmo es el que está justo encima. Dejo lo que he tenido en cuenta a continuación:
<br>

1. Construcción paso a paso (Backtracking)

* Fuerza bruta: genera todas las permutaciones de 5 dígitos y 4 operadores (362.880 expresiones completas).

* Backtracking: construye la expresión parcialmente y en cada paso decide si continuar o cortar la rama.
Esto evita llegar a expresiones completas que ya sabemos que no servirán.

2. Poda por magnitud

* Si un valor intermedio es demasiado grande (ej. >10.000 en valor absoluto), se corta la rama.

* En fuerza bruta, esa rama se exploraría completa, desperdiciando evaluaciones.

3. Poda por divisiones

* División entre cero: la rama se descarta inmediatamente.

* Divisiones no enteras cerradas, es decir, si un valor parcial ya no puede ser entero, no tiene sentido seguir expandiendo la rama.
* En fuerza bruta, estas expresiones se generarían enteras para luego descartarse al final.

4. Poda por conmutatividad

* El caso es que "a + b" y "b + a" producen el mismo valor, igual que "a x b" y "b x a".

* Imponemos un orden en el algoritmo (por ejemplo, siempre menor primero en "+" y "*") para no repetir expresiones equivalentes.

* En fuerza bruta, se calculan ambas, duplicando trabajo innecesario.

5. Evaluación anticipada (comprobaciones parciales)

* Se van evaluando las expresiones parciales para decidir si continuar. Esto aumenta el número de comprobaciones (más evaluaciones intermedias), pero reduce drásticamente el número de ramas completas exploradas.
* En fuerza bruta, solo se evalúa la expresión al final, por lo que explora mucho más.



---


###(*)Calcula la complejidad del algoritmo

**Pregunta obligatoria:**

Como hemos hecho con el algoritmo de fuerza bruta, la complejidad también varía en función de lo que entendamos por la variable $n$ de entrada.

### Opción 1: dígitos a escoger entre un conjunto de 1 a $n$

El análisis de este algoritmo de **backtracking** es más complejo que el de fuerza bruta.  
Su complejidad no se puede expresar con una fórmula simple porque depende de la estructura del árbol de búsqueda y, además, de la efectividad de las **podas**.


#### Estructura del Árbol de Búsqueda

El algoritmo construye expresiones alternando entre un dígito y un operador.  
La profundidad del árbol de búsqueda es fija: **5 niveles para los dígitos** y **4 para los operadores**.

- **Nivel 1 (dígito)**: Puedes elegir entre $n$ dígitos → $n$ ramas posibles  
- **Nivel 2 (operador)**: Puedes elegir entre 4 operadores → 4 ramas por cada rama anterior  
- **Nivel 3 (dígito)**: Puedes elegir entre los $n - 1$ dígitos restantes  
- **Nivel 4 (operador)**: Puedes elegir entre los 3 operadores restantes  
- ... y así sucesivamente hasta completar la expresión

Sin las podas, el número de nodos en este árbol de búsqueda sería enorme,  
creciendo de forma **exponencial** con $n$.

La complejidad teórica sin optimizaciones es:

$$
O(k^n)
$$

donde $k$ es el **factor de ramificación**, es decir, el número de opciones en cada paso.


#### Impacto de las Podas

Las podas son lo que vuelve eficiente este algoritmo.  
Estas condiciones detienen la exploración de una rama tan pronto como se determina que no puede llevar a una solución válida, reduciendo drásticamente el número de operaciones.

#### Poda de expresiones conmutativas:
Evita explorar ramas que son permutaciones de expresiones con operadores conmutativos (por ejemplo, $a + b$ vs $b + a$).  
Esto reduce el espacio de búsqueda por un **factor constante**.

#### Poda por valor
```python
if abs(val) > 10000:
```
Si el valor parcial es demasiado grande, la rama se corta. Poda que evita la exploración de subárboles completos.

#### Poda por división no entera
```python
if '/' in nueva_expr and not es_entero(frac):
    return
```
Elimina las ramas que ya han generado un valor no entero.

### Conclusión
Debido a que el número de ramas podadas no es una función simple de $n$, no podemos llegar a una fórmula polinómica precisa como $O(n^5)$.

La complejidad de este algoritmo de backtracking, en el peor caso, siempre es exponencial:
$$O(k^n), k=cte$$

Sin embargo, gracias a las podas, el número real de operaciones es mucho menor que en el caso teórico. El algoritmo solo explora una fracción del árbol de búsqueda total.

Por eso, en la práctica, es más rápido que el algoritmo de fuerza bruta, aunque su clasificación teórica sea superior.

### Opción 2: $n$ operadores con $n+1$ dígitos

El algoritmo construye la solución paso a paso, formando un **árbol de búsqueda**. Su complejidad, en el peor caso, corresponde al espacio de búsqueda completo, sin aplicar podas. Esta complejidad depende del número de opciones en cada etapa de construcción de la expresión.

#### Paso 1: Permutaciones de los Dígitos

El algoritmo debe elegir y ordenar $n+1$ dígitos. El número de formas de seleccionar y ordenar estos dígitos únicos es:

$$(n+1)!$$

Este término tiene un **crecimiento factorial**.

#### Paso 2: Permutaciones de los Operadores

Después de cada dígito, se elige un operador. La expresión requiere $n$ operadores. Asumiendo un conjunto suficientemente grande de operadores, el número de formas de ordenarlos es:

$$n!$$

Este término también presenta **crecimiento factorial**.

#### Paso 3: Impacto de las Podas

Las **podas** son fundamentales para la eficiencia del algoritmo. Permiten reducir drásticamente el número de nodos explorados en el árbol de búsqueda:

- **Poda por valor**: si la expresión parcial supera el objetivo.
- **Poda por división no entera**: evita operaciones inválidas.
- **Poda de expresiones conmutativas**: evita expresiones equivalentes.

Gracias a estas podas, el algoritmo evita recorrer ramas innecesarias, mejorando su rendimiento práctico.

### Conclusión

La **complejidad teórica en el peor caso** se determina por el producto de los dos términos factoriales:



$$
O((n+1)! \times n!) \approx O(n!)
$$

Se trata de una **complejidad factorial**
<br>
<br>

Aunque, para la opción 2, ambos algoritmos comparten la misma clasificación teórica, **en la práctica**, el backtracking con poda es **mucho más rápido y manejable**, ya que evita explorar combinaciones irrelevantes.

<br>
<br>

Como podemos ver, en ambas opciones, nos sale una complejidad teórica igual o peor (para el peor caso) en el del algorito de backtracking con poda.

En la práctica, esto no es real, y lo comprobamos en el apartado siguiente "Análisis comparativo de ambos algoritmos".

#### ANÁLISIS COMPARATIVO DE AMBOS ALGORITMOS

Tras varios intentos tratando de llegar a los valores esperados por el profesor, deduzco que el conteo de comprobaciones que hago con mi código comparativo es distinto al que hace el profesor. O estoy entendiendo "comprobaciones" de otra forma.
<br>

Sea como fuere, eso me ha llevado a optimizar mucho más el algoritmo de vuelta atrás con poda (que es el que he dejado en su apartado correspondiente), y conseguir resultados mejores.<br>
En este notebook dejaré solamente los resultados finales. Pero, a modo informativo, indicar que la reducción de las ramas exploradas fue cambiando; siendo inicialmente 45,10%; 77,88% con varios cambios y 92,95% en la versión final (a todo esto se le suma la mejora en el tiempo de ejecución, cuya diferencia podemos ver en la comparación siguiente).

In [ ]:
# ANÁLISIS Y COMPARACIÓN DE AMBOS ALGORITMOS

# =========================
# FUERZA BRUTA
# =========================
def fuerza_bruta():
    digitos = [str(i) for i in range(1, 10)]
    operadores = ['+', '-', '*', '/']

    expresiones_validas = []
    enteros = set()
    comprobaciones = 0

    start = time.time()

    for cifras in itertools.combinations(digitos, 5):
        for perm_cifras in itertools.permutations(cifras):
            for perm_ops in itertools.permutations(operadores):
                expr = ""
                for i in range(4):
                    expr += perm_cifras[i] + perm_ops[i]
                expr += perm_cifras[4]

                comprobaciones += 1  # cada eval final es una comprobación
                try:
                    resultado = eval(expr)
                    if resultado == int(resultado):
                        expresiones_validas.append((expr, int(resultado)))
                        enteros.add(int(resultado))
                except ZeroDivisionError:
                    continue

    tiempo = time.time() - start

    return {
        "expresiones_validas": expresiones_validas,
        "enteros": enteros,
        "tiempo": tiempo,
        "nodos": comprobaciones,  # expresiones completas exploradas
        "ramas_podadas": 0,
        "comprobaciones": comprobaciones,  # solo finales
    }


# =========================
# BACKTRACKING CON PODA
# =========================
def backtracking_con_poda_conmut():
    digitos = [str(i) for i in range(1, 10)]
    operadores = ['+', '-', '*', '/']

    expresiones_validas = []
    enteros_alcanzados = set()

    nodos_finales = 0        # expresiones completas
    comprobaciones = 0       # parciales + finales + podadas
    ramas_podadas = 0

    def es_entero(frac):
        return frac.denominator == 1

    def backtrack(expr, usados_digitos, usados_operadores, esperando_operador, ultimo_dig=None):
        nonlocal nodos_finales, comprobaciones, ramas_podadas

        # Caso base: expresión completa
        if len(usados_digitos) == 5 and len(usados_operadores) == 4:
            comprobaciones += 1
            try:
                val = eval(expr)
                if val == int(val):
                    entero = int(val)
                    enteros_alcanzados.add(entero)
                    expresiones_validas.append((expr, entero))
                    nodos_finales += 1
            except ZeroDivisionError:
                return
            return

        if esperando_operador:
            for op in operadores:
                if op in usados_operadores:
                    continue
                comprobaciones += 1  # decisión de operador
                backtrack(expr + op, usados_digitos, usados_operadores | {op}, False, ultimo_dig)

        else:
            for d in digitos:
                if d in usados_digitos:
                    continue

                # Poda 1: evitar simetrías en + y *
                if expr and expr[-1] in ['+', '*'] and ultimo_dig is not None:
                    if int(d) < int(ultimo_dig):
                        comprobaciones += 1
                        ramas_podadas += 1
                        continue

                nueva_expr = expr + d
                try:
                    comprobaciones += 1  # cada eval parcial o decisión de expandir
                    val = eval(nueva_expr)

                    # Poda 2: valor demasiado grande
                    if abs(val) > 10000:
                        ramas_podadas += 1
                        continue
                    # Poda 3: división no entera cerrada
                    frac = Fraction(val).limit_denominator()
                    if '/' in nueva_expr and not es_entero(frac):
                        ramas_podadas += 1
                        continue
                except ZeroDivisionError:
                    comprobaciones += 1
                    ramas_podadas += 1
                    continue

                backtrack(nueva_expr, usados_digitos | {d}, usados_operadores, True, d)

    start = time.time()
    backtrack("", set(), set(), False)
    tiempo = time.time() - start

    return {
        "expresiones_validas": expresiones_validas,
        "enteros": enteros_alcanzados,
        "tiempo": tiempo,
        "nodos": nodos_finales,        # expresiones completas
        "ramas_podadas": ramas_podadas,
        "comprobaciones": comprobaciones,  # todas las decisiones
    }


# =========================
# COMPARACIÓN
# =========================
print("=== FUERZA BRUTA ===")
fb = fuerza_bruta()
print(f"Tiempo: {fb['tiempo']:.2f} s")
print(f"Nodos explorados (finales): {fb['nodos']}")
print(f"Expresiones válidas: {len(fb['expresiones_validas'])}")
print(f"Enteros distintos: {len(fb['enteros'])}")
print(f"Ramas podadas: {fb['ramas_podadas']}")

print("\n=== BACKTRACKING CON PODA ===")
bt = backtracking_con_poda_conmut()
print(f"Tiempo: {bt['tiempo']:.2f} s")
print(f"Nodos explorados (finales): {bt['nodos']}")
print(f"Expresiones válidas: {len(bt['expresiones_validas'])}")
print(f"Enteros distintos: {len(bt['enteros'])}")
print(f"Ramas podadas: {bt['ramas_podadas']}")
print(f"Comprobaciones totales (parciales+finales+podadas): {bt['comprobaciones']}")

# ANÁLISIS COMPARATIVO
print("\n=== ANÁLISIS COMPARATIVO ===")
reduccion_ramas = 100 * (1 - bt['nodos'] / fb['nodos'])
incremento_comprob = 100 * (bt['comprobaciones'] / fb['comprobaciones'] - 1)

print(f"Reducción de ramas exploradas (nodos finales): {reduccion_ramas:.2f}%")
print(f"Incremento de comprobaciones: {incremento_comprob:.2f}%")


=== FUERZA BRUTA ===
Tiempo: 3.10 s
Nodos explorados (finales): 362880
Expresiones válidas: 90000
Enteros distintos: 147
Ramas podadas: 0

=== BACKTRACKING CON PODA ===
Tiempo: 0.81 s
Nodos explorados (finales): 25570
Expresiones válidas: 25570
Enteros distintos: 147
Ramas podadas: 35753
Comprobaciones totales (parciales+finales+podadas): 112305

=== ANÁLISIS COMPARATIVO ===
Reducción de ramas exploradas (nodos finales): 92.95%
Incremento de comprobaciones: -69.05%


Como se me ha indicado en la corrección, lo esperado sería:
- Reducción de ramas exploradas: 50-75%
- Incremento de comprobaciones: 200-500% (pero que se compensa con la reducción en las exploraciones)

Lo que interpreto es que:
- O bien la poda es muy agresiva, por lo que estaría descartando demasiadas ramas y haciendo menos comprobaciones (pero no tiene sentido, ya que llega a los mismos resultados finales que el algoritmo por fuerza bruta, por lo que podemos deducir que la poda "es buena").
- O bien el conteo de comprobaciones utilizado no es el esperado en un primer momento.
- O bien lo que estoy entiendo por "comprobaciones" no es lo que se está entendiendo de forma general.

De cualquier forma, queda analizada y verificada la mejora del algoritmo de backtracking con las podas explicadas en su apartado.<br>

Además, y a pesar de que teóricamente (en el peor caso), la complejidad teórica del algoritmo de backtracking es igual o peor que el algoritmo por fuerza bruta; con esta comprobación podemos verificar que el algoritmo optimizado es mejor y más eficiente:

- Llega a los mismos resultados
- Tarda mucho menos en ejecutarse
- Recorre muchas menos ramas, descartando aquellas que darían resultados no válidos o repetidos.



---


###Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

No respondida



---


###Aplica el algoritmo al juego de datos generado

No respondida



---


###Enumera las referencias que has utilizado (si ha sido necesario) para llevar a cabo el trabajo:

**Pregunta opcional:**<br>

### Bibliografía

#### Bibliografía principal
- Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). *Introduction to algorithms* (3rd ed.). MIT Press.  
  > Referencia sobre análisis de complejidad factorial, combinatoria, backtracking y branch & bound.

- Aho, A. V., Hopcroft, J. E., & Ullman, J. D. (1983). *Data structures and algorithms*. Addison-Wesley.  
  > Fundamentos sobre estructuras de datos y árboles de búsqueda, útiles para backtracking y poda.

- Knuth, D. E. (1997). *The art of computer programming, Vol. 1: Fundamental algorithms* (3rd ed.). Addison-Wesley.  
  > Explicación de permutaciones, combinatoria y análisis de algoritmos recursivos.

- Michalewicz, Z., & Fogel, D. B. (2004). *How to solve it: Modern heuristics* (2nd ed.). Springer.  
  > Métodos heurísticos y técnicas de poda para optimización.

- Eiben, A. E., & Smith, J. E. (2003). *Introduction to evolutionary computing*. Springer.  
  > Contexto sobre algoritmos evolutivos y metaheurísticas, relacionados con la optimización.

- Sedgewick, R., & Wayne, K. (2011). *Algorithms* (4th ed.). Addison-Wesley.  
  > Referencia adicional para análisis de complejidad y poda de ramas en backtracking.

- Brassard, G., & Bratley, P. (1997). *Fundamentos de algoritmia*.  
  > Referencia estándar para el análisis de algoritmos, incluyendo el conteo de operaciones elementales y la identificación de funciones de crecimiento dominantes (polinómica, exponencial, etc.).

- Lee, R. C. T., Tseng, S. S., Chang, R. C., & Tsai, Y. T. (2005). *Introducción al diseño y análisis de algoritmos*.  
  > Explicación de los conceptos de complejidad de tiempo y espacio aplicados a algoritmos de fuerza bruta y recursivos.

#### Documentación técnica de Python
- Python Software Foundation. (2024). *Python documentation*. https://docs.python.org/3/  
- Python Software Foundation. (2024). *itertools — Functions creating iterators for efficient looping*. En *Python documentation*. https://docs.python.org/3/library/itertools.html  
- Python Software Foundation. (2024). *fractions — Rational numbers*. En *Python documentation*. https://docs.python.org/3/library/fractions.html  

#### Material docente y complementario
- Universidad Internacional de Valencia. *Manual de la asignatura Algoritmos de optimización*.  
- Universidad Internacional de Valencia. *Manual de la asignatura Python para Inteligencia Artificial*.  
- Ortiz Ordóñez, J. (s. f.). *Canal de YouTube John Ortiz Ordoñez*. https://www.youtube.com/c/JohnOrtizOrdo%C3%B1ez  
- Mouredev. (s. f.). *Canal de YouTube Mouredev*. https://www.youtube.com/@mouredev  
- ChioCode. (2022). *Memoización y cómo aplicarla*. YouTube. https://www.youtube.com/watch?v=e5zaZEfHsIs




---


###Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

**Pregunta opcional:**<br>

Las formas de avanzar con este problema (en la búsqueda de mejores resultados de eficiencia y optimización) sería:

- Poda todavía más agresiva.
- Aplicar otros métodos heurísticos.
- "Memoización".
<br>

Aunque hay que tener mucho cuidado con esto, ya que si la poda es demasiado agresiva puede dar lugar a descartar opciones que darían resultados interesantes o buenos, pero que se descartan demasiado pronto; pudiendo no llegar a todos los resultados enteros.
<br>

En caso de que el problema fuese de mayor tamaño, podría interesar no llegar a los resultados exactos pero sí a resultados aproximados si ello supusiese una rapidez en la ejecución. Todo depende de lo que prime más para cada problema y cada momento, y a lo que se le quiera (o necesita) dar prioridad.



---



---


*Trabajo / Proyecto entregable*
- *Universidad Internacional de Valencia*
- *Algoritmos de optimización*
- *Borja Allo Casal*



---



---

